In [17]:
from torch_ssl import *

In [23]:
RATIO = 0.9 # train-test ratio
with open('../../data/P19_xgboost.pkl', 'rb') as f:
    dataset = pickle.load(f)
segments = dataset['data']
metadata = dataset['meta']
max_seq_len = metadata['max_seq_len']
ds_size = metadata['size']
num_sensors = metadata['num_sensors']
print(f'dataset size = {ds_size}')
print(f'max seq len in dataset = {max_seq_len}')
print(f'num sensors = {num_sensors}')
x_tr = []
x_ev = []
y_tr = []
y_ev = []
for i, segment in enumerate(segments):
    obs = segment['obs']
    label = segment['label']
    seq_len = obs.shape[0]
    obs_padded = np.pad(obs, ((0, max_seq_len-seq_len), (0, 0)), mode='constant', constant_values=0)
    if i + 1 <= int(RATIO * ds_size):
        x_tr.append(obs_padded)
        y_tr.append(label)
    else:
        x_ev.append(obs_padded)
        y_ev.append(label)
layer1 = torchSaab(kernel_size=(4,1), stride=(4,1), channelwise=False)
layer2 = torchSaab(kernel_size=(4,1), stride=(4,1), channelwise=True)
layer3 = torchSaab(kernel_size=(4,2), stride=(4,2), channelwise=True)
layer4 = torchSaab(kernel_size=(4,2), stride=(4,2), channelwise=True)
# model = sslModel([layer1, layer2, layer3, layer4])
model = sslModel([layer1, layer2, layer3, layer4])

dataset size = 40336
max seq len in dataset = 336
num sensors = 34


In [24]:
x_tr = np.expand_dims(x_tr, axis=1)  # Axis 1 is after the first dimension
print(x_tr.shape)
model.fit(x_tr)
x_tr = model(x_tr)
x_tr = x_tr.reshape(-1, 1024)
print(f'features size = {x_tr.shape}')

(36302, 1, 336, 7)
learned parameters for layer 0, output size was torch.Size([36302, 4, 84, 7])
learned parameters for layer 1, output size was torch.Size([36302, 16, 21, 7])
learned parameters for layer 2, output size was torch.Size([36302, 128, 5, 3])
learned parameters for layer 3, output size was torch.Size([36302, 1024, 1, 1])
features size = (36302, 1024)


In [25]:
classifier = LogisticRegression(max_iter=1000)
classifier.fit(x_tr, np.array(y_tr))

LogisticRegression(max_iter=1000)

In [26]:
x_ev = np.expand_dims(x_ev, axis=1)  # Axis 1 is after the first dimension
print(x_ev.shape)
x_ev = model(x_ev)
x_ev = x_ev.reshape(-1, 1024)
print(f'features size = {x_ev.shape}')


(4034, 1, 336, 7)
features size = (4034, 1024)


In [27]:
y_probs = classifier.predict_proba(x_ev)[:, 1] # probabilities for the positive class
auroc = roc_auc_score(np.array(y_ev), y_probs)
print(f"AUROC: {auroc:.4f}")

AUROC: 0.8195
